# Notebook: Create OPA2VEC associations file - Phenotypes
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Get phenotypes associated with diseases and chemicals. Create association file between entity and phenotypes, for use in opa2vec. Also output finalclasses.lst, a file that tells opa2vec which entities you would like the vectors for <br>
(i) creates map of Disease IDs to DOIDS

In [102]:
import pandas as pd
import numpy as np
import subprocess
import pickle
import ast
import re

##### Create CTD to CID map for all chemicals... 
will take ages to run, so might as well create full map instead of running each time I need it

In [103]:
# # Add PubChem CIDs to chems -- ONLY have to run this if pkl object not already made, takes a while you see
# df = pd.read_csv('allchemsthathavegenesID.txt', names=['ID'])

# # Export CTD IDs for conversion
# np.savetxt(r'CTDids.csv', df_chems.ID.unique(), fmt='%s')

# Run a pre-made script to convert them via an API --> Creates CTD:CID map as ctd_cid_map.pkl
# subprocess.check_output('python3 CTDtoCID.py allchemsthathavegenesID.txt', shell=True)


In [104]:
# This creates map as pickle object --> a python dict that you can load like so:
# # Load the dictionary object output by the above-run script
# def load_obj(name):
#     with open(name + '.pkl', 'rb') as f:
#         return pickle.load(f)

# ctd_cid_map = load_obj('ctd_cid_map')

### 1. Read in Disease and Chemical IDs
And map in the relevant ID format (Pubchem CID for chems, Disease ontology ID for Diseases)

In [105]:
# Read in all of the CTD diseases that have positive gene association(s)
diseases = pd.read_csv('../ctd-to-nt/all-diseases-w-genes-ctd.txt', names=['MESH'])

In [106]:
# Read in the ctd map of various disease identifiers
ctd_dis = pd.read_csv('../ctd-to-nt/csvs/CTD_diseases.csv', skiprows=27)
ctd_dis['AltDiseaseIDs'] = ctd_dis.AltDiseaseIDs.astype(str)
ctd_dis = ctd_dis.drop(0)

In [107]:
ctd_dis.sample(3)

,# DiseaseName,DiseaseID,AltDiseaseIDs,Definition,ParentIDs,TreeNumbers,ParentTreeNumbers,Synonyms,SlimMappings
9084,Oligohydramnios,MESH:D016104,DO:DOID:12215,A condition of abnormally low AMNIOTIC FLUID v...,MESH:D011248,C13.703.560,C13.703,NaN,Pregnancy complication
138,Achard-Thiers syndrome,MESH:C536013,nan,NaN,MESH:D010051|MESH:D014770|MESH:D048909,C04.588.322.455/C536013|C13.351.500.056.630.70...,C04.588.322.455|C13.351.500.056.630.705|C13.35...,Adenoma associated virilism of older women|Bea...,Cancer|Endocrine system disease|Signs and symp...
2546,"Chromosome 8, trisomy",MESH:C537942,nan,NaN,MESH:D014314,C23.550.210.050.750/C537942|C23.550.210.182.50...,C23.550.210.050.750|C23.550.210.182.500,Chromosome 8 duplication|Trisomy 8,Pathology (process)


In [108]:
# Isolate only rows with DOID
ctd_dis = ctd_dis[ctd_dis.AltDiseaseIDs.map(lambda x: 'DOID' in x)]

# Use regex to extract the DOID
r = re.compile('DOID:[0-9]*')

def get_doid(x, r):
    return (r.findall(x)[-1])

ctd_dis['AltDiseaseIDs'] = ctd_dis.AltDiseaseIDs.map(lambda x: get_doid(x, r))

In [109]:
# map ctd's mesh to doid
ctd_dis2doid = dict(zip(ctd_dis.DiseaseID, ctd_dis.AltDiseaseIDs))

In [110]:
diseases.sample(6)

,MESH
2885,OMIM:615473
3893,OMIM:613672
4847,OMIM:616402
5783,OMIM:616469
3309,MESH:D016883
1348,MESH:C565833


In [111]:
# # Download a doid-mesh file offered by human disease ontology in order to make a second map from this source
# url = "https://raw.githubusercontent.com/obophenotype/human-disease-ontology/master/src/experimental/align-doid-to-mesh.tsv"
# !wget -O doid_mesh.tsv $url

In [112]:
doid_mesh = pd.read_csv('doid_mesh.tsv', sep='\t', names=['DOID', 'doid-dis', 'MESH', 'mesh-dis', 'bool', 'nans']
                       , nrows=6768)
mesh_to_doid = doid_mesh.set_index('MESH').DOID.to_dict()

In [113]:
diseases['DOID'] = diseases.MESH.map(lambda x: ctd_dis2doid.get(x))
diseases['DOID2'] = diseases.MESH.map(lambda x: mesh_to_doid.get(x))
# Note I also tried the maps mona gave me but they don't seem to add anything beyond these two

In [114]:
print(diseases.DOID.nunique())
print(diseases.DOID2.nunique())

2166
1586


In [115]:
# Merge the two columns of DOIDs
diseases['DOID'] = diseases.DOID.map(lambda x: np.nan if x is None else x)
diseases.DOID.fillna(diseases.DOID2, inplace=True)
del diseases['DOID2']

In [116]:
# Export this as a csv to be imported into opa-nn
diseases.to_csv('ctd_dis_to_doid.csv', index=False)